<a href="https://colab.research.google.com/github/Onose12/AirBnB_clone/blob/main/Another_copy_of_ENV_617_Midterm_S25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENV 617 Midterm 2025
*Time: 80 minutes*

Let's analzye data from a survey in Colorado on opinions of the oil and gas industry evaluating an experiment to understand whether social media ads changed opinions of the oil & gas industry.

*Context*: We ran the following experiment:

One set of individuals were randomly selected to receive video advertisements on social media platforms (i.e. Facebook, Instagram), which highlighted the localized health risks and economic risks of oil & gas drilling. This was the **‘treatment’** group.
Another set of individuals on social media were the **‘control’** group and did not receive these ads.

Afterwards, individuals in both the treatment and control groups were asked to answer the question: “Do you believe your community is better or worse off because of the oil and gas industry?”.

Respondents selected one of the following choices:

1 - Definitely better off,
2 - Somewhat better off,
3 - Neither better nor worse off,
4 - Somewhat worse off,
5 - Definitely worse off

We will compare answers between the treatment and control groups to evaluate whether the advertisements made a difference.

**Commit your completed work back to the 'submitted' branch in your midterm repository on GitHub.**

NOTE: If you get stuck on part 2, you can complete part 3 without part 2 - move on to part 3.

Good luck!

## Part 1: Survey Data Intro and QA


In Part 1, we will load the survey data and clean it for analysis.

### 1.1: Set Up

* Import modules (return to this if necessary)


In [1]:
# Basic
import pandas as pd
import numpy as np

# Plotting
from plotnine import *
import plotnine
from mizani.formatters import scientific_format, percent_format, currency_format

# Stats
import statsmodels.api as sm
import scipy.stats as stats

* Read in the survey data into a dataframe called `df_survey`.  The survey data is available on GitHub at the link below:
```
https://raw.githubusercontent.com/smsidekick/project-sidekick/main/blihkjhdrsers.csv
```


In [2]:
link = 'https://raw.githubusercontent.com/smsidekick/project-sidekick/main/blihkjhdrsers.csv'
df = pd.read_csv(link)

### 1.2: Orient to Data

* Orient to the survey data so you understand what you are working with. Make sure you get a sense of the size of the data, the columns – and their data types, and the range of values in the data.

In [3]:
# Preview data
df.head()


,person_id,county,FIPS,treatment,total_duration_in_sec,Q1_answer_code,Q1_answer_text
0,708,Douglas County,8035,0,23,NaN,NaN
1,17337,Adams County,8001,1,78,5.0,Definitely worse off
2,25671,Adams County,8001,0,75,4.0,Somewhat worse off
3,26938,Denver County,8031,0,228,3.0,Neither better nor worse off
4,28087,Adams County,8001,0,19,NaN,NaN


In [4]:
# a sense of the size of the data, the columns – and their data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   person_id              1216 non-null   int64  
 1   county                 1216 non-null   object 
 2   FIPS                   1216 non-null   int64  
 3   treatment              1216 non-null   int64  
 4   total_duration_in_sec  1216 non-null   int64  
 5   Q1_answer_code         880 non-null    float64
 6   Q1_answer_text         880 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 66.6+ KB


In [5]:
#the range of values in the data
df.describe()

,person_id,FIPS,treatment,total_duration_in_sec,Q1_answer_code
count,1.216000e+03,1216.000000,1216.000000,1216.000000,880.000000
mean,1.022337e+07,8050.423520,0.487664,78.444079,3.440909
std,7.876390e+06,36.284052,0.500053,66.732870,1.470579
min,7.080000e+02,8001.000000,0.000000,1.000000,1.000000
25%,1.849352e+06,8031.000000,0.000000,34.000000,2.000000
50%,1.535444e+07,8041.000000,0.000000,66.000000,4.000000
75%,1.791875e+07,8077.000000,1.000000,104.000000,5.000000
max,2.002384e+07,8125.000000,1.000000,583.000000,5.000000


### 1.3: Data Checks and Cleaning.

* In this section, clean the data and store the cleaned dataframe as a new dataframe `df_clean`.

* You can start by checking for the following:
  * Are there any duplicate responses for the same individual? Take a look at the `person_id` column.
  * Are there any incomplete survey responses?
  * Do you see "survey speeders" who complete the survey too fast? Take a look at the `total_duration_in_sec` column.
* Then, check for anything else you need to feel confident in the quality and tidiness of the data for the following analysis.

* Handle any QA issues that you find in the data. Explain your decisions and actions.

In [6]:
# Check for duplicates
df[df.duplicated(keep=False)]

,person_id,county,FIPS,treatment,total_duration_in_sec,Q1_answer_code,Q1_answer_text


In [7]:
df[df['person_id'].duplicated(keep=False)]

,person_id,county,FIPS,treatment,total_duration_in_sec,Q1_answer_code,Q1_answer_text


Are there any duplicate responses for the same individual? No

In [8]:
#Are there any incomplete survey responses?
df.isna().sum()


,0
person_id,0
county,0
FIPS,0
treatment,0
total_duration_in_sec,0
Q1_answer_code,336
Q1_answer_text,336


Are there any incomplete survey responses? No

In [9]:
#survey speeders
df['total_duration_in_sec'].describe()


,total_duration_in_sec
count,1216.000000
mean,78.444079
std,66.732870
min,1.000000
25%,34.000000
50%,66.000000
75%,104.000000
max,583.000000


In [10]:
df['total_duration_in_sec'].value_counts()

,count
total_duration_in_sec,
6,23
7,20
5,20
11,18
10,17
...,...
168,1
207,1
330,1


Do you see "survey speeders" who complete the survey too fast? Take a look at the total_duration_in_sec column. Yes 23 persons completed the survey in 6 seconds.

In [11]:
#Then, check for anything else you need to feel confident in the quality and tidiness of the data for the following analysis
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   person_id              1216 non-null   int64  
 1   county                 1216 non-null   object 
 2   FIPS                   1216 non-null   int64  
 3   treatment              1216 non-null   int64  
 4   total_duration_in_sec  1216 non-null   int64  
 5   Q1_answer_code         880 non-null    float64
 6   Q1_answer_text         880 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 66.6+ KB


In [12]:
#how many NaN
df.isna().sum()

,0
person_id,0
county,0
FIPS,0
treatment,0
total_duration_in_sec,0
Q1_answer_code,336
Q1_answer_text,336


In [13]:
#drop NaN
df = df.dropna()

In [14]:
df.isna().sum()

,0
person_id,0
county,0
FIPS,0
treatment,0
total_duration_in_sec,0
Q1_answer_code,0
Q1_answer_text,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 880 entries, 1 to 1215
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   person_id              880 non-null    int64  
 1   county                 880 non-null    object 
 2   FIPS                   880 non-null    int64  
 3   treatment              880 non-null    int64  
 4   total_duration_in_sec  880 non-null    int64  
 5   Q1_answer_code         880 non-null    float64
 6   Q1_answer_text         880 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 55.0+ KB


In [17]:
#convert answer code to int64
df['Q1_answer_code'] = df['Q1_answer_code'].astype('int64')

Handle any QA issues that you find in the data. Explain your decisions and actions. I had to drop NaN because i didn't want an error coming up in the long run

### 1.4: Check Cleaned Data

* Once you've cleaned up the survey data, it's time to check that the treatment and control groups are still roughly equal - at least in so far as we have data. Answer the following questions:

  * What are the sizes of our treatment and control groups?

  * What are the top 5 counties for the treatment group? How about the control group?

  * Do you think that these look comparable?


In [38]:
df.head()

,person_id,county,FIPS,treatment,total_duration_in_sec,Q1_answer_code,Q1_answer_text
1,17337,Adams County,8001,1,78,5,Definitely worse off
2,25671,Adams County,8001,0,75,4,Somewhat worse off
3,26938,Denver County,8031,0,228,3,Neither better nor worse off
5,42671,Park County,8093,0,330,1,Definitely better off
7,48862,Denver County,8031,0,93,2,Somewhat better off


In [40]:
# Create numeric answer score
df['answer_score'] = np.where(df['Q1_answer_text'] == 'Definitely worse off', 5,
                            np.where(df['Q1_answer_text'] == 'Somewhat worse off', 4,
                                     np.where(df['Q1_answer_text'] == 'Neither better nor worse off', 3,
                                              np.where(df['Q1_answer_text'] == 'Definitely better off', 2, 1))))

df[['Q1_answer_text','answer_score']].head(5)


,Q1_answer_text,answer_score
1,Definitely worse off,5
2,Somewhat worse off,4
3,Neither better nor worse off,3
5,Definitely better off,2
7,Somewhat better off,1


In [41]:
# Check systematiclaly whether the new field is defined properly, using groupby
df.groupby('Q1_answer_text').agg(worry_check = ('answer_score','mean'))

,worry_check
Q1_answer_text,
Definitely better off,2.0
Definitely worse off,5.0
Neither better nor worse off,3.0
Somewhat better off,1.0
Somewhat worse off,4.0


In [42]:
# Create binary variable for definetly worse off
# Here, we define neutrality as having a response of "Neither better nor worse off".
df['Neutral'] = np.where(df['Q1_answer_text'] == 'Neither better nor worse off', 1, 0)

In [43]:
df[['Q1_answer_text','Neutral']].head()

,Q1_answer_text,Neutral
1,Definitely worse off,0
2,Somewhat worse off,0
3,Neither better nor worse off,1
5,Definitely better off,0
7,Somewhat better off,0


In [44]:
# Check the binary was created properly
df.groupby('Q1_answer_text').agg(worry_min = ('Neutral','max'), worry_max = ('Neutral','max'))



,worry_min,worry_max
Q1_answer_text,,
Definitely better off,0,0
Definitely worse off,0,0
Neither better nor worse off,1,1
Somewhat better off,0,0
Somewhat worse off,0,0


In [45]:
# Check the distribution of this new outcome variable.
df.head()

,person_id,county,FIPS,treatment,total_duration_in_sec,Q1_answer_code,Q1_answer_text,answer_score,Neutral
1,17337,Adams County,8001,1,78,5,Definitely worse off,5,0
2,25671,Adams County,8001,0,75,4,Somewhat worse off,4,0
3,26938,Denver County,8031,0,228,3,Neither better nor worse off,3,1
5,42671,Park County,8093,0,330,1,Definitely better off,2,0
7,48862,Denver County,8031,0,93,2,Somewhat better off,1,0


## Part 2: Survey Coverage in Colorado
In Part 2, we will visualize the geographic distribution of our (cleaned) survey responses. We'll create a map showing the bucketed number of cleaned survey responses by Colorado county.


In [52]:
#filter geographic survey response in Colorado county
df_co = df[df['county'] == 'Colorado County']
df_co.head()



,person_id,county,FIPS,treatment,total_duration_in_sec,Q1_answer_code,Q1_answer_text,answer_score,Neutral


In [ ]:

# Visualize your geographic distribution of our (cleaned) survey responses in Colorado county


### 2.1: Read in County Shapefiles

* The link below contains county shapefiles for the entire US. Read the data into a geodataframe, `df_counties`.

  ```
    https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip
```



In [55]:
! wget 'https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip'


--2025-03-05 19:24:00--  https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip
Resolving www2.census.gov (www2.census.gov)... 23.213.92.202, 2600:1408:9000:79f::208c, 2600:1408:9000:79b::208c
Connecting to www2.census.gov (www2.census.gov)|23.213.92.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘tl_2019_us_county.zip’

tl_2019_us_county.z     [    <=>             ]  75.57M   112MB/s    in 0.7s    

2025-03-05 19:24:00 (112 MB/s) - ‘tl_2019_us_county.zip’ saved [79237324]



In [56]:

! ls

sample_data  tl_2019_us_county.zip


In [57]:
#unzip the data file
! unzip tl_2019_us_county.zip

Archive:  tl_2019_us_county.zip
 extracting: tl_2019_us_county.cpg   
  inflating: tl_2019_us_county.dbf   
  inflating: tl_2019_us_county.prj   
  inflating: tl_2019_us_county.shp   
  inflating: tl_2019_us_county.shp.ea.iso.xml  
  inflating: tl_2019_us_county.shp.iso.xml  
  inflating: tl_2019_us_county.shx   


In [58]:
#Read the shapefile into a geodataframe
shape = "tl_2019_us_county.shp"
df_shape = gpd.read_file(shape)
df_shape.head()

NameError: name 'gpd' is not defined

### 2.2: Filter Geodataframe

* Filter `df_counties` to include only counties in the state of Colorado. The State FIPS code (`STATEFP`) for Colorado is `08`. Store this in a new geodataframe, `df_counties_co`.
* Check your work.

### 2.3: Summarize Survey by County

* Next, create a dataframe that aggregates the total number of survey responses by both county name and FIPS. Save this summary to a new dataframe, `df_county_survey`. (In the next step, we'll join this onto `df_counties_co`.)

* Check how many unique counties are in the data. Does that match the number of counties we expect to find in Colorado?

### 2.4: Bucket Number of Responses

* In `df_county_survey`, create a new column `N_resp_bucket` that buckets the number of survey responses to Q1 in increments of 25 (i.e. <25, 25-50, 50-75,  etc.).
* Check your work.

### 2.5: Join Survey and Geo Data

* Join `df_counties_co` and `df_county_survey` so that you end up with a geodataframe with a row for every Colorado county, with fields for the county geometries, and the number of bucketed survey responses.
* Some code that might be helpful is `df_county_survey['FIPS'] = df_county_survey['FIPS'].astype('str').str.pad(5,'left', '0')`
* Make sure to check your merge.
* Store the joined table to a new geodataframe, `df_map`.

### 2.6: Map

* Make a map of `df_map`, coloring each county by the bucketed number of survey responses.

### 2.7: Takeaways on Survey Scope


  Looking at this data, in your judgement:

  Do we have a representative sample of the entire state?

   If you don't feel like you have enough infomation to answer definitively, describe what other information might you want to more robustly inform this question.


## Part 3: Evaluate Experiment

In Part 3, we'll evaulate if the Q1 survey responses from the treatment group (who saw the ads on Facebook about the negative impacts of oil & gas drilling) were significantly different from those in the control group.

As a reminder, in the survey, question 1 asked respondents "Do you believe your community is better or worse off because of the oil and gas industry?" Respondents answered on a scale of 1 to 5, where 1 meant "Definitely better off" and 5 meant "Definitely worse off".

### 3.1: Differences between Treatment and Control

* Calculate the average `Q1_answer_code` value for the treatment group vs. the control group, for all of Colorado.

### 3.2: Statistical Test

* Answer: Is the difference you summarized in 3.1 statistically significant?

### 3.3: Interpet Results
* In a few sentences, discuss what you calculated above in 3.1. and 3.2:
  * What can we say about the results of the experiment?
  * What is one follow up question you have, or what might be a next step to understand what is going on in greater detail?